In [1]:
import torch
import numpy as np
import os
from PIL import Image
from IPython.display import display



In [2]:
with open('train/train-images-idx3-ubyte', 'rb') as train:
    magic = int.from_bytes(train.read(4), byteorder='big')
    N = int.from_bytes(train.read(4), byteorder='big')
    row_size = int.from_bytes(train.read(4), byteorder='big')
    col_size = int.from_bytes(train.read(4), byteorder='big')
    assert(magic == 2051)
    
    training_set = np.empty((N,row_size,col_size), dtype='uint8')
    
    for i in range(N):
        data = train.read(row_size*col_size)
        img = np.frombuffer(data, dtype='ubyte').reshape((row_size,col_size))
        training_set[i] = img

In [3]:
with open('train/train-labels-idx1-ubyte', 'rb') as labels:
    magic = int.from_bytes(labels.read(4), byteorder='big')
    N = int.from_bytes(labels.read(4), byteorder='big')
    assert(magic == 2049)
    
    training_labels = np.empty(N, dtype='uint8')
    
    for i in range(N):
        training_labels[i] = int.from_bytes(labels.read(1), byteorder='big')


In [4]:
with open('test/t10k-images-idx3-ubyte', 'rb') as test:
    magic = int.from_bytes(test.read(4), byteorder='big')
    test_N = int.from_bytes(test.read(4), byteorder='big')
    row_size = int.from_bytes(test.read(4), byteorder='big')
    col_size = int.from_bytes(test.read(4), byteorder='big')
    assert(magic == 2051)
    
    test_set = np.empty((test_N,row_size,col_size), dtype='uint8')
    
    for i in range(test_N):
        data = test.read(row_size*col_size)
        img = np.frombuffer(data, dtype='ubyte').reshape((row_size,col_size))
        test_set[i] = img

In [11]:
with open('test/t10k-labels-idx1-ubyte', 'rb') as labels:
    magic = int.from_bytes(labels.read(4), byteorder='big')
    test_N = int.from_bytes(labels.read(4), byteorder='big')
    assert(magic == 2049)
    
    test_labels = np.empty(test_N, dtype='uint8')
    
    for i in range(test_N):
        test_labels[i] = int.from_bytes(labels.read(1), byteorder='big')


In [5]:
sample = 2434
display(Image.fromarray(training_set[sample]))
print(training_labels[sample])

9


In [28]:
dtype = torch.double
device = torch.device("cpu")

# B is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
B, D_in, H, D_out = 64, row_size*col_size, 100, 10

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-10
epochs = 100
for e in range(int(1e20)):    
    idx = np.random.randint(0,N,B)
    x = torch.tensor(training_set[idx].reshape(B,row_size*col_size), dtype=dtype)
    y = torch.tensor(list(map(lambda i: [1 if j == i else 0 for j in range(10)], training_labels[idx])), dtype=dtype)
    
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
#     if e % 10 == 0:
#         print(y_pred)
#         print(y)
#         print(e, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    
    if e % 1000 == 0:
        print(loss)
    

228755299676.8917
188523471.97415063
67014462.28519067
15920887.307342747
47879726.219711855
12934802.361995392
11557610.52113388
25237375.137417227
9258199.117508326
22949756.479445435
20953142.840304367
4157423.7313107913
2549525.9243282517
26835794.352033176
4659673.326176863
7159391.148904309
5148014.168949939
19395402.49748879
232296.2062548622
2433562.4668889306
3832152.2303180546
4370104.025894125
4555733.443258787
3286500.5336808558
9531598.897120908
4107010.4928160254
261537.1517156988
4427752.177373571
12334202.470434451
161127.2980024192
5610163.307684738
2406013.3296949524
1810500.699064061
46482.23980237513
217093.72515524575
1055634.5256142078
10817468.98132002
16293.043993004274
3486946.89283335
648058.5948867393
599278.5134846682
5575478.625347926
1069467.8354585422
30275.69054845831
532638.7087233071
13652.838469411627
5789762.291779539
636192.170186162
400577.92362309323
86500.44613832202
3203885.4300667695
3011095.0474736723
1093220.127508064
914350.0066255985
164935

KeyboardInterrupt: 

In [ ]:
print(w1)

In [21]:
test_sample = 4700
display(Image.fromarray(test_set[test_sample]))
print(test_labels[test_sample])

9


In [29]:
x = torch.tensor(test_set[test_sample].reshape(1,row_size*col_size), dtype=dtype)
y_pred = x.mm(w1).clamp(min=0).mm(w2)
print(y_pred)
print(np.argmax(y_pred))

tensor([[ 5.2236, 10.6870,  6.5388,  8.2323,  0.4211, 11.6384,  4.1276,  0.3387,
          5.0450, -7.9075]], dtype=torch.float64)
tensor(5)


In [30]:
correct = 0
for test in range(len(test_set)):
    x = torch.tensor(test_set[test].reshape(1,row_size*col_size), dtype=dtype)
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    if np.argmax(y_pred) == test_labels[test]:
        correct += 1

In [31]:
correct / len(test_set)

0.0913